In [21]:
import polars as pl

In [39]:
import pyarrow.dataset as ds

In [40]:
files = [
    f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-{i}.parquet"
    if i > 9
    else f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-0{i}.parquet"
    for i in range(1, 12)  # Dec not available
]

In [44]:
df = pl.concat(
    [pl.read_parquet(url) for url in files]
)

In [45]:
df.estimated_size('gb')

5.18622855655849

In [46]:
df.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,"""N""",142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,"""N""",236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,"""N""",166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,"""N""",114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,"""N""",68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0


In [68]:
(
    df.with_columns(
        is_congested=pl.col('congestion_surcharge').fill_null(0) > 0
    )
    .groupby('DOLocationID')
    .agg(
        mean_fare=pl.mean('fare_amount'),
        count=pl.count(),
        congestion=pl.mean('is_congested')
    )
    .sort('count', descending=True)
)

DOLocationID,mean_fare,count,congestion
i64,f64,u32,f64
236,10.095962,1547904,0.961205
237,9.544062,1473496,0.9707
161,11.519047,1320835,0.963478
170,11.455328,1071628,0.963954
230,15.032787,1066882,0.962985
239,11.384794,1000029,0.962241
142,10.862642,988177,0.964374
141,10.476799,978456,0.963793
48,13.402571,976235,0.959376
